In [1]:
import pandas as pd
import numpy as np

three_factor_file_name = "./data/05_df_ff_info.csv"
three_factor_df = pd.read_csv(three_factor_file_name)

fundamental_file_name = "./data/funda.csv"
fundamental_df = pd.read_csv(fundamental_file_name)

monthly_stocks_file_name = "./data/monthlystocks.csv"
monthly_stocks_df = pd.read_csv(monthly_stocks_file_name)


display(three_factor_df.head())
display(fundamental_df.head())
display(monthly_stocks_df.head())

,Date,Mkt-RF,SMB,HML,RF
0,1980-01,0.0551,0.0162,0.0175,0.0080
1,1980-02,-0.0122,-0.0185,0.0061,0.0089
2,1980-03,-0.1290,-0.0664,-0.0101,0.0121
3,1980-04,0.0397,0.0105,0.0106,0.0126
4,1980-05,0.0526,0.0213,0.0038,0.0081


,GVKEY,LPERMNO,datadate,fyear,indfmt,consol,popsrc,datafmt,curcd,csho,xrd,exchg,costat,fic,mkvalt,prcc_f,sic
0,1000,25881,12/31/1975,1975.0,INDL,C,D,STD,USD,2.098,NaN,12,I,USA,NaN,4.375,3089.0
1,1000,25881,12/31/1976,1976.0,INDL,C,D,STD,USD,2.207,NaN,12,I,USA,NaN,5.750,3089.0
2,1000,25881,12/31/1977,1977.0,INDL,C,D,STD,USD,2.226,NaN,12,I,USA,NaN,9.250,3089.0
3,1001,10015,12/31/1983,1983.0,INDL,C,D,STD,USD,3.568,0.0,14,I,USA,NaN,7.250,5812.0
4,1001,10015,12/31/1984,1984.0,INDL,C,D,STD,USD,3.568,NaN,14,I,USA,NaN,3.750,5812.0


,PERMNO,date,PRC,VOL,RET,SHROUT
0,10000,12/31/1985,NaN,NaN,NaN,NaN
1,10000,01/31/1986,-4.3750,1771.0,C,3680.0
2,10000,02/28/1986,-3.2500,828.0,-0.257143,3680.0
3,10000,03/31/1986,-4.4375,1078.0,0.365385,3680.0
4,10000,04/30/1986,-4.0000,957.0,-0.098592,3793.0


Apply filters

In [2]:
# Only US companies
filter1 = fundamental_df['indfmt'] == 'INDL'
filter2 = fundamental_df['curcd'] == 'USD'
filter3 = fundamental_df['fic'] == 'USA'

# Only big exchanges
filter4 = fundamental_df['exchg'] >= 11
filter5 = fundamental_df['exchg'] <= 19

# Remove finance companies
filter6 = ((fundamental_df['sic'] < 6000) | (fundamental_df['sic'] > 6999))

# Apply filters
fundamental_df = fundamental_df[filter1 & filter2 & filter3 & filter4 & filter5 & filter6]

print(fundamental_df.shape)

(181467, 17)


Filter abnormal returns

In [3]:
# Filter out NA values in RET column
not_na = monthly_stocks_df['RET'].notna()
monthly_stocks_df = monthly_stocks_df[not_na]

# Filter out non-float values in RET column (e.g. 'C')
monthly_stocks_df["RET"] = monthly_stocks_df["RET"].apply(pd.to_numeric, errors='coerce')

# Filter out outliers
filter1 = monthly_stocks_df['RET'] > -100
filter2 = monthly_stocks_df['RET'] < 500
monthly_stocks_df = monthly_stocks_df[filter1 & filter2]


Calc market cap

In [4]:
filter1 = monthly_stocks_df['SHROUT'] > 0
monthly_stocks_df = monthly_stocks_df[filter1]

# Get most recent SHROUT if missing
monthly_stocks_df['SHROUT'].fillna(method='ffill', inplace=True)

# Fix price
monthly_stocks_df['PRC'] = abs(monthly_stocks_df['PRC'])
monthly_stocks_df['PRC'].replace(0, np.nan, inplace=True)

# Drop na
monthly_stocks_df.dropna(subset=['SHROUT', 'PRC'], inplace=True)

# Calculate market cap
monthly_stocks_df['MKT_CAP'] = monthly_stocks_df['PRC'] * monthly_stocks_df['SHROUT']

In [5]:
monthly_stocks_df.shape

(3459593, 7)

Parse date column and shift datadate by 3 months to avoid lookahead bias

In [6]:
# Fundamental data parse datadate column
fundamental_df['datadate'] = pd.to_datetime(fundamental_df['datadate'], format='%m/%d/%Y')

# Shift it forward by 3 months
fundamental_df['datadate'] = fundamental_df['datadate'] + pd.DateOffset(months=3)

# Create a year column
fundamental_df['year'] = fundamental_df['datadate'].dt.year

XRD calculation

In [7]:
# Fill xrd column with 0 if it is NA
fundamental_df['xrd'].fillna(0, inplace=True)

# Sort the dataframe
fundamental_df.sort_values(by=['datadate', 'LPERMNO'], inplace=True)
fundamental_df.reset_index(drop=True, inplace=True)

def calc_weighted_avg(five_period_series):
    # xt * 1 + xt-1 * 0.8 + xt-2 * 0.6 + xt-3 * 0.4 + xt-4 * 0.2
    return five_period_series[4] * 1 + five_period_series[3] * 0.8 + five_period_series[2] * 0.6 + five_period_series[1] * 0.4 + five_period_series[0] * 0.2

# Calculate 5-period average of xrd
rdc = fundamental_df.groupby('LPERMNO')['xrd'].rolling(5).apply(calc_weighted_avg, raw=True)
rdc.reset_index(level=0, drop=True, inplace=True)
rdc.sort_index(inplace=True)

fundamental_df['rdc'] = rdc

fundamental_df.dropna(subset=['rdc'], inplace=True)

Split firms into R&D firms 

In [8]:
non_rd_firms = fundamental_df[fundamental_df['rdc'] == 0].copy()
rd_firms = fundamental_df[fundamental_df['rdc'] > 0].copy()

In [9]:
print("The shape of the dataframes are {} and {}".format(non_rd_firms.shape, rd_firms.shape))
print("The shape of fundamental_df is {}".format(fundamental_df.shape))

The shape of the dataframes are (61209, 19) and (61083, 19)
The shape of fundamental_df is (122292, 19)


Drop the useless columns

In [10]:
# Only keep columns LPERMNO, datadate, rdc
non_rd_firms = non_rd_firms[['LPERMNO', 'datadate', 'rdc']]
rd_firms = rd_firms[['LPERMNO', 'datadate', 'rdc']]
fundamental_df = fundamental_df[['LPERMNO', 'datadate', 'rdc']]

Most returns are end of month but some are a few days before. Fix this

In [11]:
monthly_stocks_df['date'] = pd.to_datetime(monthly_stocks_df['date'], format='%m/%d/%Y')
monthly_stocks_df['date'] = monthly_stocks_df['date'] + pd.tseries.offsets.MonthEnd(0)

Add a date Reconstitution date column in fundamental data

In [12]:
import datetime

# If data date before last day of reconstitution month, then it is the reconstitution date
# Else it is the reconstitution date of the next year

def get_reconstitution_date(row):
    reconstitution_year = row['datadate'].year
    reconstitution_month = 3
    reconstituion_day = 31
    reconst_date = datetime.datetime(reconstitution_year, reconstitution_month, reconstituion_day)

    if row['datadate'] <= reconst_date:
        return reconst_date
    else:
        return datetime.datetime(reconstitution_year + 1, reconstitution_month, reconstituion_day)

rd_firms["reconstitution_date"] = rd_firms.apply(get_reconstitution_date, axis=1)
non_rd_firms["reconstitution_date"] = non_rd_firms.apply(get_reconstitution_date, axis=1)

Merge with fundamental data with returns to add a market cap column

In [13]:
# Merge fundamental data with monthly stock data to add market cap column

# RD firms
rd_firms = pd.merge(rd_firms, monthly_stocks_df, left_on=['LPERMNO', 'reconstitution_date'], right_on=['PERMNO', 'date'], how='left')

# Non RD firms
non_rd_firms = pd.merge(non_rd_firms, monthly_stocks_df, left_on=['LPERMNO', 'reconstitution_date'], right_on=['PERMNO', 'date'], how='left')

Calculate R&D to Market cap ratio

In [20]:
rd_firms["rdc_to_mkt_cap"] = rd_firms["rdc"] / rd_firms["MKT_CAP"]
non_rd_firms["rdc_to_mkt_cap"] = non_rd_firms["rdc"] / non_rd_firms["MKT_CAP"]

Drop Nan Market cap values

In [22]:
# Drop if market cap is nan
rd_firms.dropna(subset=['MKT_CAP'], inplace=True)
non_rd_firms.dropna(subset=['MKT_CAP'], inplace=True)

In [23]:
# Export data to pkl files

rd_firms.to_pickle("data/rd_firms.pkl")
non_rd_firms.to_pickle("data/non_rd_firms.pkl")
fundamental_df.to_pickle("data/fundamental_df.pkl")
monthly_stocks_df.to_pickle("data/monthly_stocks_df.pkl")

Bad pipe message: %s [b's}y\x86-\xb6\xd4\xb0\xf2\xf7\xf1\xa0A\xf2$QB\xc1 \xb0OwS\x93\xf6\xbd\xeb\xd7O\x9d\x1f\xda\x0b\xf1\x814\xc2\xe9\x10\x81\x06\x8f\xe8\xbe\xdeM\xecr\xdb\xfdi\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07', b'\x08\t\x08\n\x08\x0b\x08']
Bad pipe message: %s [b'\x05\x08\x06']
Bad pipe message: %s [b'\x05\x01\x06', b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \x03\xe7\x1c\xbb\t\xee\x07\x99\x0e0y/\x85\x7f\x9e\xa9I\x19\xa43\xeao']
Bad pipe message: %s [b'\t\xcdy\x84\xd0\xba', b'7\\\xed\x02<@[\x04S\xd7\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f']
Bad pipe message: %s [b"\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0